## 8. Összetettebb programok

Az elmúlt hetekben megismert eszközök - ciklusok, fájlkezelés - már elegendőek ahhoz, hogy komplex feladatokat is programokkal végezzünk el. Ebben a notebook-ban azt nézzük meg, hogyan tudunk egyszerűbb, általánosabb elemekből felépíteni egy összetettebb programot.

A következő példákban ismét egy filmes adatbázissal fogunk dolgozni, ebben azonban már sokkal több különböző információ található, pl. hogy hány kritikát írtak a filmről, vagy hogy az egyes színészeknek hány lájkjuk van a facebook-on. A következőkben az alábbi kérdésekre fogunk az adat segítségével választ adni:

* Melyik az 5 legjobb film az IMDB pontszám alapján?
* Melyik 5 színésznek van a legtöbb lájkja facebook-on?
* A legtöbb FB lájkot kapó 10 film szereplői mennyi lájkot kaptak?
* A legrosszabb IMDB pontszámú filmek között milyen műfajból van a legtöbb?

### Előkészületek

Elsőként vizsgáljuk meg az adatot, amivel dolgozni fogunk. A táblázat a "data/movie_data.csv" fájlban található, mezőit vesszők választják el, az első sor a fejléc, ebből látszik, hogy melyik oszlop milyen információt tartalmaz. Nézzük meg, milyen oszlopokból áll az adatunk!

In [9]:
f = open("data/movie_data.csv", 'r', encoding="utf-8")

In [11]:
header = f.readline().strip().split(',')

In [12]:
print(header)

['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']


Olvassuk be az adatot úgy, hogy minden _oszlop_ alkosson külön listát, tehát pl. egy listában szerepeljen az összes cím, egy másikban az összes rendező, stb.

In [13]:
def adatot_beolvas(fajlnev):
    f = open(fajlnev, 'r', encoding="utf-8")
    fejlec = f.readline().strip().split('\t')
    mezok_szama = len(fejlec)
    adat = []
    for i in range(mezok_szama):
        adat.append([])
    print('mezok szama:', mezok_szama)
    for sor in f:
        mezok = sor.strip('\n').split('\t')
        for i in range(mezok_szama):
            adat[i].append(mezok[i].strip())

    return adat, fejlec        

In [14]:
adat, fejlec = adatot_beolvas("data/movie_data.tsv")

mezok szama: 28


A _fejlec_ listából megtudhatjuk, hogy valamely infó hanyadik oszlopban található. Nézzük meg, az első 5 filmnek a listában ki a főszereplője?

In [21]:
fejlec.index("actor_1_name")

10

In [22]:
adat[10][:5]

['CCH Pounder', 'Johnny Depp', 'Christoph Waltz', 'Tom Hardy', 'Doug Walker']

Először tudni szeretnénk az IMDB pontszám alapján 5 legjobb film címét. Írjunk egy olyan függvényt, ami tetszőleges mező szerint rendezve képes visszaadni az adatot.

In [78]:
def rendez(adat, fejlec, mezo):
    adat_meret = len(adat[0])
    oszlop_szam = fejlec.index(mezo)
    rendezendo = []
    ertekek = adat[oszlop_szam]
    for i in range(adat_meret):
        rendezendo.append([ertekek[i], i])
    rendezendo.sort()
    rendezendo.reverse()
    rendezett_sorszamok = []
    for ertek, i in rendezendo:
        rendezett_sorszamok.append(i)
    return rendezett_sorszamok

Ez a függvény egy tetszőleges mező szerint rendezve visszaadja az egyes filmek sorszámait.

In [28]:
pontszam_szerint = rendez(adat, fejlec, "imdb_score")

Egy második függvényt használunk majd arra, hogy adott sorszámú mezőkből adja vissza egy másik mező értékét:

In [29]:
def keres(adat, fejlec, sorszamok, mezo):
    oszlop_szam = fejlec.index(mezo)
    ertekek = []
    for i in sorszamok:
        ertekek.append(adat[oszlop_szam][i])
    return ertekek

Írjunk ezek segítségével egy olyan függvényt, ami valamely mező szerint rendezve visszaadja egy másik mező értékeit:

In [55]:
def elsok(adat, fejlec, kulcs_mezo, ertek_mezo, n):
    rendezett_sorszamok = rendez(adat, fejlec, kulcs_mezo)
    ertekek = keres(adat, fejlec, rendezett_sorszamok[:n], ertek_mezo)
    return ertekek

Az első kérdésre a választ ezután az alábbi paranccsal tudjuk lekérni:

In [52]:
elsok(adat, fejlec, "imdb_score", "movie_title", 5)

['Towering Inferno',
 'The Shawshank Redemption',
 'The Godfather',
 'Kickboxer: Vengeance',
 'Dekalog']

A második kérdésre ("_Melyik 5 színésznek van a legtöbb lájkja facebook-on?_") ez a megoldás már nem fog működni:

In [53]:
elsok(adat, fejlec, "actor_1_facebook_likes", "actor_1_name", 5)

['Gary Cooper', 'Gary Cooper', 'Sofia Boutella', 'Chris Brown', 'Chris Brown']

Olyan adatra van szükségünk, ami az egyes színészeket listázza a lájkjaik számával:

In [59]:
def szineszeket_listaz(adat, fejlec):
    adat_meret = len(adat[0])
    szinesz_oszlop = fejlec.index("actor_1_name")
    lajk_oszlop = fejlec.index("actor_1_facebook_likes")
    szineszek = []
    lajkok = []
    for i in range(adat_meret):
        szinesz = adat[szinesz_oszlop][i]
        if szinesz not in szineszek:
            szineszek.append(szinesz)
            lajkok.append(adat[lajk_oszlop][i])
    return [szineszek, lajkok]

In [60]:
szineszek = szineszeket_listaz(adat, fejlec)

In [69]:
szineszek[0][:5]

['CCH Pounder', 'Johnny Depp', 'Christoph Waltz', 'Tom Hardy', 'Doug Walker']

Ha ehhez az adathoz is készítünk fejlécet, akkor már használhatjuk hozzá az _elsok_ függvényünket!

In [63]:
szinesz_fejlec = ["actor_1_name", "actor_1_facebook_likes"]

In [79]:
elsok(szineszek, szinesz_fejlec, "actor_1_facebook_likes", "actor_1_name", 5)

['Gary Cooper',
 'Sofia Boutella',
 'Chris Brown',
 'Bérénice Bejo',
 'Hector Elizondo']

A harmadik feladathoz (_A legtöbb FB lájkot kapó 10 film szereplői mennyi lájkot kaptak?_) először keressük meg a legtöbb lájkot kapott 10 filmet a már meglévő függvényeinkkel:

In [81]:
elsok(adat, fejlec, "movie_facebook_likes", "movie_title", 10)

['The Blood of Heroes',
 'Assault on Precinct 13',
 'Freedom',
 'Roll Bounce',
 "Hart's War",
 'L!fe Happens',
 'Red Planet',
 'The Big Short',
 'The Wood',
 'Sabrina, the Teenage Witch']

Ezután írjunk egy olyan függvényt, ami adott értékeket keres egy-egy oszlopban és összegyűjti egy másik oszlopból a hozzá tartozó más adatokat. A _keres_ függvényt újra fogjuk tudni hasznosítani:

In [83]:
def ertekhez_keres(adat, fejlec, kulcs_mezo, kulcsok, ertek_mezo):
    kulcs_oszlop = adat[fejlec.index(kulcs_mezo)]
    sorszamok = []
    for kulcs in kulcsok:
        sorszam = kulcs_oszlop.index(kulcs)
        sorszamok.append(sorszam)
    
    ertekek = keres(adat, fejlec, sorszamok, ertek_mezo)
    return ertekek

In [85]:
top_lajkolt_filmek = elsok(adat, fejlec, "movie_facebook_likes", "movie_title", 10)

In [86]:
ertekhez_keres(adat, fejlec, "movie_title", top_lajkolt_filmek, "cast_total_facebook_likes")

['1526', '3148', '994', '7552', '15916', '2924', '2', '57308', '2724', '2433']

És ha kiváncsiak vagyunk, hogy milyen színészek hozták ezt össze, azt is könnyen megnézhetjük:

In [87]:
ertekhez_keres(adat, fejlec, "movie_title", top_lajkolt_filmek, "actor_1_name")

['Delroy Lindo',
 'Brian Dennehy',
 'Sharon Leal',
 'Jurnee Smollett-Bell',
 'Bruce Willis',
 'Justin Kirk',
 'Bob Neill',
 'Ryan Gosling',
 'Omar Epps',
 'Nate Richert']

In [88]:
ertekhez_keres(adat, fejlec, "movie_title", top_lajkolt_filmek, "actor_2_name")

['Joan Chen',
 'Drea de Matteo',
 'David Rasche',
 'Brandon T. Jackson',
 'Cole Hauser',
 'Geoff Stults',
 'Val Kilmer',
 'Christian Bale',
 'Tamala Jones',
 'Soleil Moon Frye']

In [89]:
ertekhez_keres(adat, fejlec, "movie_title", top_lajkolt_filmek, "actor_3_name")

['Anna Katarina',
 'Hugh Dillon',
 'Bart Shatto',
 'Shad Moss',
 'Rory Cochrane',
 'Kristen Johnston',
 'Tom Sizemore',
 'Charlie Talbert',
 'Richard T. Jones',
 'Caroline Rhea']

__8.1. FELADAT__ Írj olyan függvényt, ami egy megadott mező értékeit átlagolja! Átlagosan hány lájkot kapott egy film a Facebook-on?

__8.2. FELADAT__ Az IMDB pontszám alapján 100 legjobb film átlagosan hány lájkot kapott?

__8.3. FELADAT__ Módosítsd az _elsok_ függvényt úgy, hogy ha n negatív, akkor a sorszámokat a lista végéről vegye, vagyis az elsők helyett az utolsókat listázza! Az IMDB pontszám alapján 100 legrosszabb film átlagosan hány lájkot kapott?

__8.4. FELADAT (Pluszpontokért beadható! Határidő: 2016. november 9. 08:00)__ A 100 legrosszabb IMDB pontszámú film között milyen műfajból van a legtöbb?